In [ ]:
#!eval-notebook "../Initialization/InitSystemorphToMemory"

In [ ]:
var scenario = "IRU";
var year = 2020;
var month = 12;
var reportingNode = "CH";
var dataNode = "DT1.1";
var format = ImportFormats.Cashflow;

In [ ]:
Workspace.Initialize(x => x.FromSource(DataSource));

In [ ]:
var targetPartition = (await DataSource.Query<PartitionByReportingNodeAndPeriod>().Where(p => p.ReportingNode == reportingNode &&
                                                                                                   p.Year == year &&
                                                                                                   p.Month == month &&
                                                                                                   p.Scenario == scenario).ToArrayAsync()).Single().Id;
var defaultPartition = (await DataSource.Query<PartitionByReportingNodeAndPeriod>().Where(p => p.ReportingNode == reportingNode &&
                                                                                                    p.Year == year &&
                                                                                                    p.Month == month &&
                                                                                                    p.Scenario == null).ToArrayAsync()).Single().Id;


# Import Scenario Data

In [ ]:
//await UploadCashflowsAsync("../Files/900.TransactionalData/NominalCashflows_CH_2020_12_Scenario.csv")
var fileName = "../Files/900.TransactionalData/NominalCashflows_CH_2020_12_Scenario.csv";
var args = (await GetArgsFromMainAsync<PartitionByReportingNodeAndPeriod>(fileName)) with {ImportFormat = ImportFormats.Cashflow};
await DataNodeFactoryAsync(fileName, ImportFormats.Cashflow, args);
    
var parsingLog = await ParseCashflowsToWorkspaceAsync(fileName, args);
var targetPartition = (await DataSource.Query<PartitionByReportingNodeAndPeriod>().Where(p => p.ReportingNode == args.ReportingNode &&
                                                                                                   p.Year == args.Year &&
                                                                                                   p.Month == args.Month &&
                                                                                                   p.Scenario == args.Scenario).ToArrayAsync()).Single().Id;
await Workspace.Partition.SetAsync<PartitionByReportingNodeAndPeriod>(targetPartition);                                                                                            
await DataSource.Partition.SetAsync<PartitionByReportingNodeAndPeriod>(targetPartition);
var parsedVariables = await Workspace.Query<RawVariable>().ToArrayAsync();
var savedVariables = await DataSource.Query<RawVariable>().ToArrayAsync();
if (!parsedVariables.Any()) return true;
var updatedVariables = savedVariables?.Union(parsedVariables, RawVariableComparer.Instance) ?? parsedVariables;
var newVariables = updatedVariables?.Except(parsedVariables, RawVariableComparer.Instance) ?? Enumerable.Empty<RawVariable>();
updatedVariables

In [ ]:
await UploadCashflowsAsync("../Files/900.TransactionalData/NominalCashflows_CH_2020_12_Scenario_BE.csv")

In [ ]:
Workspace.InitializeFrom(DataSource);